# Activity: La mañanera

Con esta actividad se pretende conocer una manera artesanal de hacer análisis de lenguaje natural. 

También sirve el propósito de introducirnos a python.

In [ ]:
import wget
import re
import glob
import io
from bs4 import BeautifulSoup
from datetime import datetime
import os
import urllib.request

In [ ]:
url_pagina_i = "https://presidente.gob.mx/secciones/version-estenografica/page/2/"
#url_pagina_i = "http://www.futurecrew.com/skaven/song_files/mp3/razorback.mp3"
wget.download(url = url_pagina_i, out = "../data/mananera/pagina_i.txt")

In [ ]:
url = "https://presidente.gob.mx/secciones/version-estenografica/page/2/"

In [ ]:
response = urllib.request.urlopen(url)
webContent = response.read().decode('UTF-8')
f = open('obo-t17800628-33.html', 'w')
f.write(webContent)
f.close

In [ ]:
import requests
r = requests.get(url, allow_redirects=True)
open('facebook.ico', 'wb').write(r.content)

In [ ]:
pattern = '(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/-+#-]*[\w@?^=%&/~+#-])?'
re.findall(pattern, open("./pagina_i.txt", "r", encoding = "utf-8").read())

In [ ]:
url_pagina_i = "https://presidente.gob.mx/secciones/version-estenografica/page/"
num_paginas = 69
for i in range(num_paginas):
    wget.download(url = url_pagina_i + str(i + 1) + "/", out = "../data/mananera/" + str(i + 1) + ".txt")

In [ ]:
urls_conf = []

for f_pagina in glob.glob("../data/mananera/*"):
    print(f_pagina)
    urls_conf = [path for _, _, path in re.findall(re.findall(pattern, open(f_pagina, "r", encoding = "utf-8").read())) if "estenografica-de-la" in path]

In [ ]:
urls_conf_set = list(set(urls_conf))

In [ ]:
for path in urls_conf_set:
    f = wget.download(url = "https://presidente.gob.mx" + path, out = "./estenograficas/" + path.replace("/", "-"))[0]    

In [ ]:
for f_paginas in glob.glob("../data/mananera/estenograficas/*"):
    soup = BeautifulSoup(open(f_pagina, "r", encoding = "utf-8").read(), 'html.parser')
    open("./estenograficas_limpias/" + f_pagina.replace("./estenograficas/", ""), "w", encoding = "utf-8").write(soup.get_text())

In [ ]:
def normaliza(str_fecha):
    datetimeobject = datetime.strptime(str_fecha[:6] + "20" + str_fecha[6:], '%d.%m.%Y') 
    newformat = datetimeobject.strftime('%Y-%m-%d')
    return newformat

In [ ]:
i = 0
fechas_conferencia = []
for f_pagina in glob.glob("./estenograficas_limpias/*"):
    texto = open(f_pagina, 'r', encoding = 'utf-8').read()
    fechas_en_archivo = re.findall('[0-9][0-9]\.[0-9][0-9]\.[0-9][0-9]', texto)
    print(fechas_en_archivo)
    if len(fechas_en_archivo) == 5:
        if not os.path.isfile('./estenograficas_limpias_por_fecha/' + normaliza(fechas_en_archivo[0])):
            open("./estenograficas_limpias_por_fecha/" + normaliza(fechas_en_archivo[0]), 'w', encoding = "utf-8").write(texto)
            fechas_conferencia += [normaliza(fechas_en_archivo[0])]
        elif not os.path.isfile('./estenograficas_limpias_por_fecha/' + normaliza(fechas_en_archivo[0]) + "_2da"):
            open("./estenograficas_limpias_por_fecha/" + normaliza(fechas_en_archivo[0]) + "_2da", 'w', encoding = "utf-8").write(texto)
            print("Segunda:", f_pagina)
            fechas_conferencia += [normaliza(fechas_en_archivo[0])]
        elif not os.path.isfile('./estenograficas_limpias_por_fecha/' + normaliza(fechas_en_archivo[0]) + "_3ra"):
            open("./estenograficas_limpias_por_fecha/" + normaliza(fechas_en_archivo[0]) + "_3ra", 'w', encoding = "utf-8").write(texto)
            fechas_conferencia += [normaliza(fechas_en_archivo[0])]
            print("Tercera:", f_pagina)
    else:
        print("Raro:", f_pagina)